<a href="https://colab.research.google.com/github/Vishwajeet0830/Financial-Agent/blob/main/AI_Hiring_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
!pip install -qU openai-agents==0.0.3 pydantic pdfplumber rich python-dotenv


In [65]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")


Enter your OpenAI API key: ··········


In [66]:
from google.colab import files

uploaded = files.upload()  # Upload the PDF file again
resume_pdf_path = list(uploaded.keys())[0]  # Get the uploaded file name
print(f"✅ Uploaded File: {resume_pdf_path}")


Saving Vishwajeet_Niradi_DA.pdf to Vishwajeet_Niradi_DA.pdf
✅ Uploaded File: Vishwajeet_Niradi_DA.pdf


In [67]:
# Extract text from the uploaded resume
resume_text = extract_text_from_pdf("Vishwajeet_Niradi_DA.pdf")

# Print the first 4000 characters for debugging
print(resume_text[:4000])


VISHWAJEET NIRADI
Montclair, NJ| (862) 409-0121 |niradiv1@montclair.edu |LinkedIn| GitHub
EDUCATION
Montclair State University, Montclair, NJ May 2025
Master of Science in Data Science 4.0 GPA
Coursework: Exploratory Data Analysis and Visualization, Data Management and Database Design, Data Science and Machine Learning
Systems Engineering, Big Data Analytics
SDM College of Engineering and Technology, India Aug 2021
Bachelor of Engineering in Computer Science and Engineering
Coursework: Statistics, Python Programming, Engineering Mathematics, Probability, Machine Learning, Database Management Systems, Object
Oriented Programming
TECHNICAL SKILLS
Programming Languages: Python, R, C/C++
Business intelligence: Tableau, Power BI, Alteryx
Analytical Techniques: AB testing, DAX, Hypothesis testing, t-test, Chi-squared
Statistical Techniques: Regression, Classification, Clustering, A/B tests
Database Systems: SQL, MySQL, PostgreSQL
Libraries / Frameworks: Pandas, NumPy, Matplotlib, Seaborn, Py

/usr/local/lib/python3.11/dist-packages/pdfplumber/utils/clustering.py:61: RuntimeWarning: coroutine 'get_job_fit_score' was never awaited
  ((x, cluster_dict.get(key_fn(x))) for x in xs), key=get_1


In [68]:
!pip install -q rich pandas
import pandas as pd
import re

def extract_candidate_info(resume_text):
    """Extracts candidate details (name, skills, full experience) from resume."""

    # Extract Name (First Line in ALL CAPS)
    name_match = re.match(r'^[A-Z][A-Z\s]+', resume_text.strip())
    candidate_name = name_match.group().strip() if name_match else "Unknown"

    # Extract Skills (Search for TECHNICAL SKILLS section)
    skills_section = re.search(r"TECHNICAL SKILLS([\s\S]+?)PROFESSIONAL EXPERIENCE", resume_text, re.IGNORECASE)
    skills = []
    if skills_section:
        skills = re.findall(r"([A-Za-z0-9+/\-#]+)", skills_section.group(1))

    # Extract FULL Work Experience (until next major section)
    experience_section = re.search(r"PROFESSIONAL EXPERIENCE([\s\S]+?)(PROJECTS|CERTIFICATIONS|EDUCATION)",
                                   resume_text, re.IGNORECASE)
    experience = experience_section.group(1).strip() if experience_section else "Unknown"

    # Clean bullet points and formatting issues
    formatted_experience = experience.replace("\uf0b7", "-").replace("•", "-")

    return {
        "name": candidate_name,
        "skills": skills[:20],  # Limit to top 20 skills
        "experience": formatted_experience  # Full experience section extracted
    }

# Extract structured data from the resume
candidate_info = extract_candidate_info(resume_text)

# Display the results
def display_candidate_info(candidate_info):
    """Formats and displays extracted resume details in a structured way."""
    print("\n🎯 Candidate Resume Summary 🎯")
    print("=" * 50)

    # Display Name
    candidate_name = candidate_info['name'].splitlines()[0]
    print(f"👤 Name: {candidate_name}")

    # Display Skills
    if candidate_info["skills"]:
        skills_df = pd.DataFrame({"Skills": candidate_info["skills"]})
        print("\n📌 Extracted Skills:")
        display(skills_df)
    else:
        print("\n📌 Extracted Skills: No skills detected.")

    # Display Full Experience
    print("\n📂 Full Work Experience:")
    print(candidate_info["experience"])

# Call function to display extracted details
display_candidate_info(candidate_info)




🎯 Candidate Resume Summary 🎯
👤 Name: VISHWAJEET NIRADI

📌 Extracted Skills:


,Skills
0,Programming
1,Languages
2,Python
3,R
4,C/C++
5,Business
6,intelligence
7,Tableau
8,Power
9,BI



📂 Full Work Experience:
Gully2Global Oct 2022 – Apr 2023
Data Analyst Bengaluru, India
- Managed end-to-end project lifecycle for 10+ Tableau dashboards—coordinating timelines, resources, and cross-functional teams—to
improve client data comprehension by 40% and drive actionable insights for critical business challenges.
- Integrated SQL databases with Tableau for major clients, resolving complex data integration issues and improving data reporting
efficiency by 20% leveraging Python to perform advanced data analysis to derive insights and recommend business strategies.
- Streamlined data processing workflows through automation and process improvements, boosting processing speed by 20%, and
presented data-driven insights to stakeholders, enhancing decision-making processes.
Accenture Sep 2021 – Aug 2022
Full Stack Engineering Analyst Bengaluru, India
- Within an Agile environment, oversaw sprint planning, resource allocation, and risk mitigation, reducing project durations by 15%
w

In [69]:
!pip install nest_asyncio


In [70]:
import asyncio
import nest_asyncio  # Fix async issues in Jupyter Notebooks
from typing import List
from agents import Agent, Runner, function_tool

# Apply nest_asyncio for Jupyter/Colab compatibility
nest_asyncio.apply()

@function_tool
def job_match(candidate_skills: List[str], job_skills: List[str]) -> float:
    """Matches candidate skills with job requirements and returns a fit score."""
    common_skills = set(candidate_skills) & set(job_skills)  # Find matching skills
    score = (len(common_skills) / len(job_skills)) * 100 if job_skills else 0
    return round(score, 2)

# Define an agent to call the function tool
job_match_agent = Agent(
    name="Job Match AI",
    instructions="Match candidate skills to job requirements and return a score.",
    model="gpt-4o-mini",
    tools=[job_match]  # Register the function as a tool
)

async def get_job_fit_score(candidate_skills: List[str], job_skills: List[str]):
    """Calls the AI tool to compute job fit score."""
    response = await Runner.run(
        starting_agent=job_match_agent,
        input=[  # Correct format: list of dicts for conversation-style input
            {"role": "user", "content": f"Match candidate skills {candidate_skills} with job requirements {job_skills}"}
        ]
    )
    return response.final_output  # Extracts AI-generated response

# Run the async function properly in Jupyter Notebook
job_fit_score = asyncio.run(get_job_fit_score(candidate_info["skills"], job_requirements))

print(f"\n📈 Job Fit Score: {job_fit_score}%")



📈 Job Fit Score: The match score between the candidate skills and job requirements is **50.0**. This indicates a moderate level of alignment between the skills of the candidate and the needs of the job.%


In [71]:
def job_match_local(candidate_skills, job_skills):
    """Simulate AI job matching without OpenAI API."""
    common_skills = set(candidate_skills) & set(job_skills)  # Find matching skills
    score = (len(common_skills) / len(job_skills)) * 100 if job_skills else 0
    return round(score, 2)

# Define job requirements (Modify this as needed for different jobs)
job_requirements = ["Python", "SQL","R","testing", "Machine Learning", "Power BI", "Tableau", "PyTorch"]


In [72]:
# Print extracted skills & job requirements before matching
print(f"\n📌 Candidate Skills: {candidate_info['skills']}")
print(f"📌 Job Requirements: {job_requirements}")

# Run job matching locally
job_fit_score = job_match_local(candidate_info["skills"], job_requirements)

# Display structured job match results
print("\n📈 Job Fit Score:")
print(f"✅ The candidate skills match the job requirements with a score of {job_fit_score}%.")

# Show which skills matched
matched_skills = list(set(candidate_info["skills"]) & set(job_requirements))
print(f"🔍 Matched Skills: {matched_skills}")

# Show missing skills
missing_skills = list(set(job_requirements) - set(candidate_info["skills"]))
print(f"❌ Missing Skills: {missing_skills}")



📌 Candidate Skills: ['Programming', 'Languages', 'Python', 'R', 'C/C++', 'Business', 'intelligence', 'Tableau', 'Power', 'BI', 'Alteryx', 'Analytical', 'Techniques', 'AB', 'testing', 'DAX', 'Hypothesis', 'testing', 't-test', 'Chi-squared']
📌 Job Requirements: ['Python', 'SQL', 'R', 'testing', 'Machine Learning', 'Power BI', 'Tableau', 'PyTorch']

📈 Job Fit Score:
✅ The candidate skills match the job requirements with a score of 50.0%.
🔍 Matched Skills: ['Tableau', 'Python', 'testing', 'R']
❌ Missing Skills: ['Power BI', 'SQL', 'Machine Learning', 'PyTorch']


In [73]:
def run_interview_interactive(candidate_info):
    """Conducts an interactive interview where the candidate types answers."""
    print(f"\n🗣️ Interviewing: {candidate_info['name']}...\n")

    questions = [
        "Tell me about your experience in data analysis.",
        f"What is your expertise in {candidate_info['skills'][0]}?",
        "Describe a challenging project you worked on and how you solved problems.",
        "Can you explain a time when you had to analyze a large dataset?",
        "How do you approach troubleshooting errors in SQL queries?",
        "What are some key insights you've generated from data that impacted a business decision?"
    ]

    responses = []
    for question in questions:
        print(f"\n📝 Question: {question}")
        answer = input("✍️ Your Answer: ")  # Take manual input from the candidate
        responses.append(answer)

    return responses

# Run interactive interview
interview_responses = run_interview_interactive(candidate_info)



🗣️ Interviewing: VISHWAJEET NIRADI
M...


📝 Question: Tell me about your experience in data analysis.
✍️ Your Answer: I am well versed in data analysis

📝 Question: What is your expertise in Programming?
✍️ Your Answer: I am proficient with Python and R

📝 Question: Describe a challenging project you worked on and how you solved problems.
✍️ Your Answer: I worked on my LLM powered QA application where i had to fine tune many iterations to finally avoid hallucinations

📝 Question: Can you explain a time when you had to analyze a large dataset?
✍️ Your Answer: Yeah in my British Airways prediction model i had to analyze all the customer reviews and then had to scrape it from web and then clean it and then perform adata analysis

📝 Question: How do you approach troubleshooting errors in SQL queries?
✍️ Your Answer: Not sure

📝 Question: What are some key insights you've generated from data that impacted a business decision?
✍️ Your Answer: I worked on my British airways prediction model 

In [74]:
from typing import List
from agents import function_tool, Agent, Runner
import asyncio
import nest_asyncio

# Apply nest_asyncio to allow multiple loops in Jupyter
nest_asyncio.apply()

@function_tool
def evaluate_interview_responses(responses: List[str]) -> dict:
    """
    Uses AI to score candidate interview responses and provide recommendations.
    Returns:
    - Score (0-100%)
    - Feedback on weak areas
    - Improvement tips
    """
    total_score = 0
    feedback = []  # Store improvement suggestions

    for response in responses:
        response_score = 0
        suggestion = ""

        # Scoring logic
        if len(response) > 200:
            response_score = 10  # Well-explained answers score highest
        elif 100 < len(response) <= 200:
            response_score = 7  # Medium-length answers
        else:
            response_score = 5  # Short answers get lowest score

        # Generate feedback based on response
        if len(response) < 150:
            suggestion = "Your response could be more detailed. Consider adding specific examples or technical depth."
        elif "I don't know" in response or "Not sure" in response:
            suggestion = "Try to provide an attempt at an answer, even if uncertain. This shows problem-solving skills."
        elif "SQL" in response and "optimize" not in response:
            suggestion = "For SQL questions, mention how you optimize queries or handle large datasets for better clarity."

        total_score += response_score
        if suggestion:
            feedback.append(f"🔹 **Issue:** {suggestion}")

    # Normalize score to 100%
    final_score = min(100, round((total_score / (len(responses) * 10)) * 100, 2))

    return {
        "score": final_score,
        "feedback": feedback[:3] if feedback else ["No major improvements needed."]
    }


In [75]:
scoring_agent = Agent(
    name="Scoring AI",
    instructions="Evaluate candidate interview responses and return a percentage score.",
    model="gpt-4o-mini",
    tools=[evaluate_interview_responses]  # Register function as an AI tool
)


In [76]:
import re  # Regular expressions to extract numbers

async def get_interview_score(responses: List[str]):
    """Uses OpenAI's Agents SDK to evaluate interview responses."""

    # Ensure responses are properly formatted
    structured_input = [{"role": "user", "content": f"Candidate's response: {response}"} for response in responses]

    # Run the AI function
    response = await Runner.run(
        starting_agent=scoring_agent,
        input=structured_input
    )

    # Extract only the numeric score using regex
    match = re.search(r"(\d+(\.\d+)?)%", response.final_output)  # Extracts "78.33%"
    score = float(match.group(1)) if match else 0  # Convert to float

    return score  # Return numerical score

# Run the AI-powered scoring function
interview_score = asyncio.run(get_interview_score(interview_responses))

# Display the formatted score
print(f"\n📊 **AI-Generated Interview Score: {interview_score:.2f}%**")


ERROR:openai.agents.tracing:Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: sk-proj-********************************************************************************************************************************************************XsEA. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}



📊 **AI-Generated Interview Score: 60.00%**


In [77]:
def generate_report(candidate, job_fit, interview, final, matched, missing, feedback):
    """Generates a structured AI hiring report with improvement recommendations."""
    report = f"""
    🎯 **Candidate Report**
    ---------------------------
    📌 **Name:** {candidate['name']}
    🛠 **Skills:** {', '.join(candidate['skills'])}

    📈 **Job Fit Score:** {job_fit}%
    🗣️ **AI-Generated Interview Score:** {interview}%
    🎯 **Final Score:** {final}%

    🔍 **Matched Skills:** {', '.join(matched)}
    ❌ **Missing Skills:** {', '.join(missing)}
    """

    if feedback:
        report += "\n\n📌 **Areas for Improvement & Recommendations:**\n"
        for tip in feedback:
            report += f"🔹 {tip}\n"

    report += f"\n✅ **Status:** {'Recommended' if final >= 75 else 'Needs Improvement'}"

    return report


In [78]:
# Generate final structured report
final_report = generate_report(
    candidate_info,
    job_fit_score,
    interview_score,
    (job_fit_score * 0.5) + (interview_score * 0.5),  # Final Score
    matched_skills,
    missing_skills,
    improvement_tips  # AI-generated feedback
)

# Display the final report
print(final_report)



    🎯 **Candidate Report**
    ---------------------------
    📌 **Name:** VISHWAJEET NIRADI
M
    🛠 **Skills:** Programming, Languages, Python, R, C/C++, Business, intelligence, Tableau, Power, BI, Alteryx, Analytical, Techniques, AB, testing, DAX, Hypothesis, testing, t-test, Chi-squared
    
    📈 **Job Fit Score:** 50.0%
    🗣️ **AI-Generated Interview Score:** 60.0%
    🎯 **Final Score:** 55.0%
    
    🔍 **Matched Skills:** Tableau, Python, testing, R
    ❌ **Missing Skills:** Power BI, SQL, Machine Learning, PyTorch
    

📌 **Areas for Improvement & Recommendations:**
🔹 Error: AI response not in expected format.

✅ **Status:** Needs Improvement
